In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sheet\\Desktop\\MachineLearning\\Briefly\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sheet\\Desktop\\MachineLearning\\Briefly'

In [5]:
from dataclasses import dataclass     #this goes in the entity file
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    ALL_REQUIRED_FILES: list
    


In [6]:
import sys
import os

sys.path.append(os.path.abspath("src"))

from Briefly.constants import *  # Now it should work
from Briefly.utils.common import  read_yaml, create_directories
# this goes in configuration

In [ ]:
class ConfigurationManager:           #This goes into configuration.py
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        """Initialize Configuration Manager and load YAML configs."""
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Ensure artifacts directory exists
        create_directories([self.config["artifacts_root"]])   #this part is same in ingestion and validation
        
        
     #only copy this part in configuration.py for validation   
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            ALL_REQUIRED_FILES=config.ALL_REQUIRED_FILES,
        )
        return data_validation_config
        
        
        

In [8]:
import os
from Briefly.logging import logger


In [16]:
import os

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        
    def validate_all_files_exist(self) -> bool:
        try:
            validation_status = True  # Assume all files exist initially

            # Get all files in the dataset directory
            dataset_path = os.path.join("artifacts", "data_ingestion", "samsum_dataset")
            all_files = os.listdir(dataset_path)

            # Check if any required file is missing
            for required_file in self.config.ALL_REQUIRED_FILES:
                if required_file not in all_files:
                    validation_status = False  # If any file is missing, mark as False
                    break  # Exit early if a file is missing

            # Write the validation result to the status file
            with open(self.config.STATUS_FILE, "w") as f:
                f.write(f"validation status: {validation_status}")

            return validation_status

        except Exception as e:
            raise e


In [17]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config= data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2025-03-31 13:32:38,597: INFO: yaml file: config\config.yaml loaded successfully]
[2025-03-31 13:32:38,602: INFO: yaml file: params.yaml loaded successfully]
[2025-03-31 13:32:38,604: INFO: created directory at: artifacts]
[2025-03-31 13:32:38,606: INFO: created directory at: artifacts/data_validation]
